In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array,load_img
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf
from glob import glob
import warnings
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import os
import cv2

warnings.filterwarnings("ignore")

# read data

In [ ]:
train_dir = '/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Training/'
test_dir = '/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test/'
val_dir='/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Validation/'

# visualize some data 

In [ ]:
def create_df(path):
    # creating a list of paths to all the files with the .jpg extension.
    filepaths = list(glob(path+'**/*.jpg'))
    
    # creating a list of paths to all the files with the .jpg extension.
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
    
    # creating a DataFrame from the lists of filepaths and labels.
    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    df = pd.concat([filepaths, labels], axis=1)
    
    return df


In [ ]:
train = create_df(train_dir)
print('Train shape is ' , train.shape)

In [ ]:
test = create_df(val_dir)
print('Val shape is ' , test.shape)

In [ ]:
val = create_df(test_dir)
print('Test shape is ' , val.shape)

In [ ]:
random_img = train.sample(100).reset_index()
fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(25, 25),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(random_img.Filepath[i]))
    ax.set_title(random_img.Label[i], color='blue', fontsize=14)
    ax.axis('off')
plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
img=load_img(train_dir+"apple_crimson_snow_1/r0_0.jpg")
plt.figure(figsize=(2,2))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
img=load_img(train_dir+"eggplant_violet_1/r0_0.jpg")
plt.figure(figsize=(2,2))
plt.imshow(img)
plt.axis("off")
plt.show()

# using simple CNN model

In [ ]:
x=img_to_array(img)
x.shape

In [ ]:
className=glob(train_dir+"/*")
print(className[3],len(className))

In [ ]:
model = tf.keras.Sequential()
#first layer of model
model.add(layers.Conv2D(32,(3,3),input_shape=x.shape))
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))
#second layer of model
model.add(layers.Conv2D(64,(3,3)))
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))





model.add(layers.Flatten())
model.add(layers.Dense(1024))#input neurons
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(className)))#output layer
model.add(layers.Activation("softmax"))

In [ ]:
model.summary()

# data_augmentation

In [ ]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,   #grey images
    shear_range=0.3,  #rotating images randomly
    horizontal_flip=True,  #randomly flip image 90 degree
    zoom_range=0.3
)
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=x.shape[:2],
    color_mode="rgb",
    class_mode="categorical"
)
val_generator=test_datagen.flow_from_directory(
    val_dir,
    target_size=x.shape[:2],
    color_mode="rgb",
    class_mode="categorical"
)

test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=x.shape[:2],
    color_mode="rgb",
    class_mode="categorical"
)

## compile the model 

In [ ]:
optimizer=tf.keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    epsilon=1e-07,
    name="RMSprop"
)

In [ ]:
#Compile Model
model.compile(optimizer = optimizer, loss='categorical_crossentropy', 
             metrics=["accuracy"])

## fit model

In [ ]:
batch_size=32

In [ ]:
history=model.fit_generator(generator=train_generator,
                   steps_per_epoch=1600//batch_size,#first get our images,then look for generated images till 1600 is done
                   epochs=10,
                   validation_data=val_generator,
                   validation_steps=800//batch_size
                   )

In [ ]:
def prepare_data(train):
    data = []
    labels = []
    for row in train.itertuples():
        img = cv2.imread(row.Filepath)
        img = cv2.resize(img, (15, 15))
        img = img.astype('float32') / 255.0
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        data.append(img.flatten())
        labels.append(row.Label)

    data = np.asarray(data)
    labels = np.asarray(labels)

    return data, labels

x_train, y_train = prepare_data(train)
x_test, y_test = prepare_data(test)
x_val, y_val = prepare_data(val)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_test = le.fit_transform(y_test)
class_names = np.unique(le.inverse_transform(y_test))

In [ ]:
test_batch = next(test_generator)
images = test_batch[0][:4]
labels = test_batch[1][:4]


predictions = np.argmax(model.predict(images), axis=1);


fig, axs = plt.subplots(1, 4, figsize=(15, 5))
for i in range(4):
    img = images[i]
    label = class_names[np.argmax(labels[i])]
    pred = class_names[predictions[i]]
    axs[i].imshow(img)
    axs[i].set_title(f'Real: {label}\nPredicted: {pred}')
    axs[i].axis('off')
plt.show()

## visualize the result

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
#plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# using VGG TL Model

## reshape input to 224*224 to suitable with vgg input layer

In [ ]:
train_data=ImageDataGenerator().flow_from_directory(train_dir,target_size=(224,224))
val_data=ImageDataGenerator().flow_from_directory(val_dir,target_size=(224,224))
test_data=ImageDataGenerator().flow_from_directory(test_dir,target_size=(224,224))

## import vgg_model

In [ ]:
vgg= VGG16()


In [ ]:
vgg.summary()

In [ ]:
print(len(vgg.layers))

## freezing layers

In [ ]:
vgg.trainable = False


In [ ]:
vgg.summary()

## remove the prediction layer of vgg then add our prediction layer

In [ ]:
model_vgg = tf.keras.Sequential()
for i in range(len(vgg.layers)-1):
    model_vgg.add(vgg.layers[i])
print(model_vgg.summary())

In [ ]:
from tensorflow.keras import layers
model_vgg.add(layers.Dense(len(className),activation="softmax"))

In [ ]:
model_vgg.summary()

## compile vgg model

In [ ]:
model_vgg.compile(loss="categorical_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

## fit model

In [ ]:
hist=model_vgg.fit_generator(train_data,steps_per_epoch=1600//batch_size,
                        epochs=5,
                        validation_data=val_data,
                        validation_steps=800//batch_size)

## visualize the result

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

loss = hist.history['loss']
val_loss = hist.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
#plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_batch = next(test_data)
images = test_batch[0][:4]
labels = test_batch[1][:4]


predictions = np.argmax(model_vgg.predict(images), axis=1);


fig, axs = plt.subplots(1, 4, figsize=(15, 5))
for i in range(4):
    img = images[i]
    label = class_names[np.argmax(labels[i])]
    pred = class_names[predictions[i]]
    axs[i].imshow(img)
    axs[i].set_title(f'Real: {label}\nPredicted: {pred}')
    axs[i].axis('off')
plt.show()

## fine tune some layers

In [ ]:
model_vgg.trainable = True

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 17

# Freeze all the layers before the `fine_tune_at` layer
for layer in model_vgg.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
model_vgg.compile(loss="categorical_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

In [ ]:
model_vgg.summary()

In [ ]:
hist=model_vgg.fit_generator(train_data,steps_per_epoch=1600//batch_size,
                        epochs=5,
                        validation_data=val_data,
                        validation_steps=800//batch_size)

## visualize the result

In [ ]:
acc = hist_.history['accuracy']
val_acc = hist_.history['val_accuracy']

loss = hist_.history['loss']
val_loss = hist_.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
#plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# using ResNet

In [ ]:
train_df = pd.DataFrame(train)
train_df.reset_index(drop=True, inplace=True)
train_df.info()

In [ ]:
test_df = pd.DataFrame(test)
test_df.reset_index(drop=True, inplace=True)
test_df.info()

In [ ]:
val_df = pd.DataFrame(val)
val_df.reset_index(drop=True, inplace=True)
val_df.info()

In [ ]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
    )
def progress_image(df, shuffle):
    image = generator.flow_from_dataframe(
        dataframe=df,
        x_col='Filepath',
        y_col='Label',
        target_size=(644, 143),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=shuffle,
    )
    return image


train_images = progress_image(train_df, True)
val_images = progress_image(test_df, False)
test_images = progress_image(val_df, False)

In [ ]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input

In [ ]:
x.shape

In [ ]:
resnet_model = tf.keras.applications.resnet50.ResNet50(input_shape=x.shape,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
resnet_model.trainable = False


In [ ]:
resnet_model.summary()

In [ ]:
x = resnet_model.output

# Add a global average pooling layer
x = GlobalAveragePooling2D()(x)

# Add your custom dense layer
predictions = Dense(24, activation='softmax')(x)  

# Create the final model
model_res = Model(inputs=resnet_model.input, outputs=predictions)

In [ ]:
model_res.summary()

In [ ]:
model_res.compile(loss="categorical_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

In [ ]:
hist_res=model_res.fit_generator(train_images,steps_per_epoch=1600//batch_size,
                        epochs=5,
                        validation_data=val_images,
                        validation_steps=800//batch_size)

## visualize result of resNet

In [ ]:
acc = hist_res.history['accuracy']
val_acc = hist_res.history['val_accuracy']

loss = hist_res.history['loss']
val_loss = hist_res.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
#plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_batch = next(test_images)
images = test_batch[0][:4]
labels = test_batch[1][:4]


predictions = np.argmax(model_res.predict(images), axis=1);


fig, axs = plt.subplots(1, 4, figsize=(15, 5))
for i in range(4):
    img = images[i]
    label = class_names[np.argmax(labels[i])]
    pred = class_names[predictions[i]]
    axs[i].imshow(img)
    axs[i].set_title(f'Real: {label}\nPredicted: {pred}')
    axs[i].axis('off')
plt.show()

## fine tune some layers

In [ ]:
print("Number of layers in the base model: ", len(model_res.layers))

In [ ]:
model_res.trainable = True

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 140

# Freeze all the layers before the `fine_tune_at` layer
for layer in model_res.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model_res.summary()

In [ ]:
model_res.compile(loss="categorical_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

In [ ]:
hist_res_fine= model_res.fit_generator(train_data,steps_per_epoch=1600//batch_size,
                        epochs=10,
                        initial_epoch=hist_res.epoch[-1],
                        validation_data=test_data,
                        validation_steps=800//batch_size)

## vusualize result

In [ ]:
acc = hist_res_fine.history['accuracy']
val_acc = hist_res_fine.history['val_accuracy']

loss = hist_res_fine.history['loss']
val_loss = hist_res_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
#plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## using augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
x = resnet_model.output
x = data_augmentation(x)
# Add a global average pooling layer
x = GlobalAveragePooling2D()(x)

# Add your custom dense layer
predictions = Dense(24, activation='softmax')(x)  

# Create the final model
model_res_aug = Model(inputs=resnet_model.input, outputs=predictions)

In [ ]:
model_res_aug.compile(loss="categorical_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

In [ ]:
hist_res_aug= model_res_aug.fit_generator(train_data,steps_per_epoch=1600//batch_size,
                        epochs=3,
                        #initial_epoch=hist_res.epoch[-1],
                        validation_data=test_data,
                        validation_steps=800//batch_size)

In [ ]:
acc = hist_res_aug.history['accuracy']
val_acc = hist_res_aug.history['val_accuracy']

loss = hist_res_aug.history['loss']
val_loss = hist_res_aug.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
#plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()